In [ ]:
# Andrew Dunn, Katherine Dumais, Kathryn Link-Oberstar, Lee-Or Bentovim
# Initial exploratory analysis for Costa Rican household poverty level prediction

# This analysis runs after we load, clean, and generate additional variables in load_data.py

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
from load_data import load_train_data 
import json
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt


### Load Data

In [ ]:
# Load data
X_train, X_valid, y_train, y_valid = load_train_data()

# combine the training datasets into one
df = X_train.merge(y_train, left_index = True, right_index = True)

# Load variable descriptions
with open('var_descriptions.json', 'r') as f:
    # Load JSON data as a dictionary
    var_desc = json.load(f)

In [ ]:
len(df.loc[:, 'parentesco1'].unique())

### Data Cleaning
- There are a few features which are coded as strings but mix strings and integers/floats (see example below):
    - 'dependency', 
    - 'edjefe', 
    - 'edjefa'
    - 'tamviv'

We will need to decide how to handle these.

### Lit review
Below are notes from research papers we consulted. The focus on variables we may create and tips on how we may do our analysis.

[Understanding the Determinants of Poverty](https://web.worldbank.org/archive/website01407/WEB/IMAGES/PMCH8.PDF)

- using the highest level of the individuals in the household as the 
household level characteristic. IE, education level of the most highly educated
person in the household

[Introduction to Poverty Analysis](https://documents1.worldbank.org/curated/en/775871468331250546/pdf/902880WP0Box380okPovertyAnalysisEng.pdf)

- p88 - use household head characteristics

[HOUSEHOLD CHARACTERISTICS AND POVERTY: A LOGISTIC REGRESSION ANALYSIS](https://www.jstor.org/stable/23612271?seq=8)

- p310
    - use presence of disability, able-bodied persons, in the household
    - sex ratio in household
    - child/woman ratio in household
    - proportion of female workers to total workers
    - dependency ratio

[Understanding poverty through household and individual level characteristics](https://worldbank.github.io/SARMD_guidelines/note-hhdchars.html)

- "For example, it is not true in general that female-headed households have lower levels of expenditures per capita"
- "It is true, however, that urban households have significantly higher expenditures per capita"

[The DHS Wealth Index](https://dhsprogram.com/pubs/pdf/cr6/cr6.pdf)

- "For this reason, Filmer and Pritchett recommended using principal components analysis
(PCA) to assign the indicator weights, the procedure that is used for the DHS wealth index."

[Poverty and its measurement](https://www.ine.es/en/daco/daco42/sociales/pobreza_en.pdf)

- p8-9 - calculate income per consumption unit rather than per capita

[ARE POOR INDIVIDUALS MAINLY FOUND IN POOR HOUSEHOLDS? EVIDENCE USING NUTRITION DATA FOR AFRICA](https://www.nber.org/system/files/working_papers/w24047/w24047.pdf)

[Moving from the Household to the Individual: Multidimensional Poverty Analysis](https://arxiv.org/ftp/arxiv/papers/1304/1304.5816.pdf)
- "Using longitudinal data Medeiros and Costa (2008) conclude that
feminisation of poverty has not occurred in the eight Latin American countries they
studied. Their findings are invariant to different measures and definitions of poverty."
- "marital status is an important consideration when discussing poverty incidence"

### Data Exploration

First, we see that the Target variable is not consistently distributed. There are far more values of 4 than there are of 1, which may present issues for classification.

Below, we ran a bivariate regression for each feature (except for 'dependency', 'edjefe', and 'edjefa' - see note about need for data cleaning above) on the Target, filtered out results not significnat at the 5% level and returned:

* the top 5 features by r-squared
    
* the top 5 features by the absolute value of the coefficient

In [ ]:
features_to_include = [x for x in var_desc.keys() if x not in ['Id', 'idhogar', 'dependency', 'rez_esc', 'hh_head', 'parentesco1', 'hh_head_exists']]
# CAN WE REMOVE DEPENDENCY FROM THIS LIST AFTER THE CLEANING?
# WHY IS REZ_ESC ALL NA?


df_subset = df[features_to_include]
df_subset 

In [ ]:
# Check the variable data types
df_subset.dtypes.value_counts() 
g = df.columns.to_series().groupby(df.dtypes).groups
g

In [ ]:
df_subset.isna().sum().sort_values()

In [ ]:
# Replace inifinity with NA
#df_subset.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill in NaN with the column mean
df_subset = df_subset.fillna(df_subset.mean())
# df_subset = df.copy()

# Select the target column and the other columns of interest & exclude the 3 columns discussed above with mixed datatypes 
target_col = 'Target'
other_cols = [x for x in df_subset.columns if x not in ['Target']]
results_df = pd.DataFrame(columns=['variable', 'coefficient', 'p_value', 'r_squared'])

# Iterate over each independent variable in the dataframe
for col in df_subset.columns[:-1]:
    # Fit a linear regression model on the independent variable and target
    X = df_subset[[col]]
    y = df_subset[target_col]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    
    # Get the coefficient, p-value, and R-squared for the model
    coeff = model.params[1]
    p_value = model.pvalues[1]
    r_squared = model.rsquared
    
    results_df.loc[len(results_df)] = [col, coeff, p_value, r_squared]

results_df['variable_desc'] = results_df['variable'].map(var_desc)
print(results_df)

In [ ]:
# Filter results where p_value is less than or equal to 0.05 and sort by r-squared
results_df = results_df[results_df['p_value'] <= 0.05]
results_df = results_df.sort_values(by='r_squared', ascending = False)
print(results_df.head(5))

In [ ]:
# Filter results where p_value is less than or equal to 0.05 and sort by coefficient
results_df = results_df[results_df['p_value'] <= 0.05]
results_df = results_df.iloc[abs(results_df['coefficient']).argsort()[::-1]] 
print(results_df.head(5))

In [ ]:
results_df.to_csv('regression_results.csv', index=False)

In [ ]:
# Comparison of individual and household level characteristics

In [ ]:
Individual = ['v18q', 'tamviv', 'escolari', 'dis', 'male', 'female', 'estadocivil1','estadocivil2','estadocivil3','estadocivil4','estadocivil5','estadocivil6',\
'estadocivil7','parentesco2','parentesco3','parentesco4', 'parentesco5', 'parentesco6', 'parentesco7',\
'parentesco8', 'parentesco9', 'parentesco10', 'parentesco11' ,'parentesco12', 'instlevel1', 'instlevel2', 'instlevel3', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9', 'mobilephone']
# 'parentesco1', 'rez_esc',

Household = [
    'v2a1','hacdor',
'rooms', 'hacapo','v14a', 'refrig', 'v18q1', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4m2','r4m3','r4t1', 'r4t2','r4t3','tamhog','hhsize',
'paredblolad','paredzocalo','paredpreb','pareddes','paredmad','paredzinc','paredfibras','paredother','pisomoscer','pisocemento',
'pisoother','pisonatur','pisonotiene','pisomadera','techozinc', 'techoentrepiso', 'techocane', 'techootro','cielorazo','abastaguadentro',
'abastaguafuera','abastaguano', 'public','planpri','noelec','coopele','sanitario1','sanitario2','sanitario3','sanitario5','sanitario6','energcocinar1',
'energcocinar2','energcocinar3','energcocinar4','elimbasu1','elimbasu2','elimbasu3','elimbasu4','elimbasu5','elimbasu6','epared1','epared2','epared3','etecho1',
'etecho2','etecho3','eviv1','eviv2','eviv3',#'idhogar',
'hogar_nin','hogar_adul','hogar_mayor','hogar_total', 'edjefe','edjefa', #'dependency',
'meaneduc','bedrooms','overcrowding','tipovivi1','tipovivi2','tipovivi3','tipovivi4','tipovivi5','computer','television','qmobilephone',
'lugar1','lugar2','lugar3','lugar4','lugar5','lugar6',
'area1','area2']

If we split on these we learn the following:

In [ ]:
df_subset2 = df_subset.copy() 
df_subset = df_subset[Household].fillna(df_subset[Household].mean())

# Select the target column and the other columns of interest
target_col = "Target"
other_cols = Household

# Create an empty dataframe to store the regression results
results_df = pd.DataFrame(columns=["variable", "coefficient", "p_value", "r_squared"])

# Iterate over each independent variable in the dataframe
for col in df_subset.columns[:-1]:
    # Fit a linear regression model on the independent variable and target
    X = df_subset[[col]]
    y = y_train["Target"]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    # Get the coefficient, p-value, and R-squared for the model
    coeff = model.params[1]
    p_value = model.pvalues[1]
    r_squared = model.rsquared

    # Add the results to the results dataframe
    results_df.loc[len(results_df)] = [col, coeff, p_value, r_squared]

# Add a column with the variable descriptions
results_df["variable_desc"] = results_df["variable"].map(var_desc)

# Filter the results where p_value is less than or equal to 0.05
results_df = results_df[results_df["p_value"] <= 0.05]

# Sort the results by r-squared from least to greatest
results_df = results_df.sort_values(by="r_squared", ascending=False)

# Print the results dataframe
print(results_df)

In [ ]:
df_subset = df_subset2.copy()
df_subset = df_subset[Individual].fillna(df_subset[Individual].mean())

# Select the target column and the other columns of interest
target_col = "Target"
other_cols = Individual 

# Create an empty dataframe to store the regression results
results_df = pd.DataFrame(columns=["variable", "coefficient", "p_value", "r_squared"])

# Iterate over each independent variable in the dataframe
for col in df_subset.columns[:-1]:
    # Fit a linear regression model on the independent variable and target
    X = df_subset[[col]]
    y = y_train["Target"]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    # Get the coefficient, p-value, and R-squared for the model
    coeff = model.params[1]
    p_value = model.pvalues[1]
    r_squared = model.rsquared

    # Add the results to the results dataframe
    results_df.loc[len(results_df)] = [col, coeff, p_value, r_squared]

# Add a column with the variable descriptions
results_df["variable_desc"] = results_df["variable"].map(var_desc)

df_subset=df_subset2.copy()

# Filter the results where p_value is less than or equal to 0.05
results_df = results_df[results_df["p_value"] <= 0.05]

# Sort the results by r-squared from least to greatest
results_df = results_df.sort_values(by="r_squared", ascending=False)

# Print the results dataframe
print(results_df)

This demonstrates that on an individual level the Household characteristics provide a lot more explanitory value than the individual characteristics. Understanding that poverty does not exist in a vacuum-- the conditions of someones family and environment are important factors in understanding their financial status. 

In [ ]:
# Correlation of variables with Target
no_target_df, _, target, _ = load_train_data()
df = no_target_df.copy()
df['Target'] = target
corrs = abs(no_target_df.corrwith(df.loc[:,'Target'], method='spearman')).sort_values(ascending=False)
corrs.head(10)


In [ ]:
# Define functions to plot data
def plot_continuous(df, continuous_col, target_col, cutoff=None):
    """
    Plot target distribution by continuous variable
    
    Parameters:
    df (pandas.DataFrame): Dataframe containing both continuous and target variables
    continuous_col (str): Name of the continuous variable column
    target_col (str): Name of the target variable column
    cutoff (float): Optional cutoff value for the continuous variable
    
    Returns:
    None
    """
    df[continuous_col] = df[continuous_col].fillna(df[continuous_col].mean())
    
    # Cap continuous column at cutoff if specified
    if cutoff:
        df.loc[df[continuous_col] > cutoff, continuous_col] = cutoff
    
    # Define the bins using the distribution of the continuous variable
    if cutoff:
        bins = [df[continuous_col].min(), df[continuous_col].quantile(0.25), df[continuous_col].median(), df[continuous_col].quantile(0.75), cutoff]
    else:
        bins = np.percentile(df[continuous_col], [0, 25, 50, 75, 100])
    bins = np.unique(bins)
    # Create a label for each bin, handling duplicates if necessary
    labels = []
    for i in range(len(bins)-1):
        label = f'{int(bins[i])}-{int(bins[i+1])}'
        count = labels.count(label)
        if count > 0:
            label = f'{label} ({count})'
        labels.append(label)
    
    df[f'{continuous_col}_bin'] = pd.cut(df[continuous_col], bins=bins, labels=labels, include_lowest=True, right=True, duplicates='drop')
    
    # Count target variable by continuous variable bin
    target_by_bin = df.groupby(f'{continuous_col}_bin')[target_col].value_counts(normalize=True).unstack().fillna(0)
    
    # Define the x-axis tick labels
    tick_labels = labels
    
    # Plot the bars
    fig, ax = plt.subplots()
    bar_width = 0.2
    opacity = 0.8
    
    for i in range(len(target_by_bin.columns)):
        rects = ax.bar(np.arange(len(tick_labels))+i*bar_width, target_by_bin.iloc[:,i], bar_width, alpha=opacity, label=f'Target = {target_by_bin.columns[i]}')
    
    # Add axis labels, title, and legend
    ax.set_xlabel(continuous_col)
    ax.set_ylabel('Percentage')
    ax.set_title(f'Target Distribution by {continuous_col}')
    ax.set_xticks(np.arange(len(tick_labels))+0.5*(len(target_by_bin.columns)-1)*bar_width)
    ax.set_xticklabels(tick_labels)
    ax.legend()
    
    plt.tight_layout()
    plt.show()

def plot_grouped_bar_chart(df, binary_col, target_col):
    """
    Plots a grouped bar chart showing the percentage of each target category for each value of a binary column.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data.
    binary_col (str): The name of the binary column.
    target_col (str): The name of the target column.

    Returns:
    None
    """

    # Create a grouped bar chart
    fig, ax = plt.subplots()
    bar_width = 0.35
    opacity = 0.8

    # Get the unique values of the binary column
    index = df[binary_col].unique()

    # Count target variable by binary group
    binary_yes = df[df[binary_col] == 1][target_col].value_counts().sort_index().tolist()
    binary_no = df[df[binary_col] == 0][target_col].value_counts().sort_index().tolist()

    # Calculate percentage of total for each target category within each binary group
    binary_yes_perc = [count/sum(binary_yes) * 100 for count in binary_yes]
    binary_no_perc = [count/sum(binary_no) * 100 for count in binary_no]
    binary_yes_target_perc = [binary_yes[i]/(binary_yes[i]+binary_no[i]) * 100 for i in range(len(binary_yes))]
    binary_no_target_perc = [binary_no[i]/(binary_yes[i]+binary_no[i]) * 100 for i in range(len(binary_no))]

    # Define the x-axis tick labels
    tick_labels = sorted(df[target_col].unique())

    # Plot the bars
    rects1 = ax.bar([i - bar_width/2 for i in range(len(tick_labels))], binary_yes_target_perc, bar_width, alpha=opacity, color='b', label=f'{binary_col}=Yes')
    rects2 = ax.bar([i + bar_width/2 for i in range(len(tick_labels))], binary_no_target_perc, bar_width, alpha=opacity, color='orange', label=f'{binary_col}=No')

    # Add axis labels, title, and legend
    ax.set_xlabel(target_col)
    ax.set_ylabel('Percentage of Total')
    ax.set_title(f'{target_col} count by {binary_col}')
    ax.set_xticks([i for i in range(len(tick_labels))])
    ax.set_xticklabels(tick_labels)
    ax.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot variables with the highest correlation with Target and other variables of interest

In [ ]:
plot_continuous(df, 'meaneduc', 'Target', cutoff=21)

In [ ]:
plot_continuous(df, 'SQBmeaned', 'Target', cutoff=441)

In [ ]:
plot_continuous(df,'v2a1_log','Target')

In [ ]:
plot_continuous(df,'SQBdependency','Target')

In [ ]:
plot_continuous(df,'escolari','Target')

In [ ]:
plot_continuous(df,'hogar_nin','Target')

In [ ]:
plot_grouped_bar_chart(df, 'cielorazo', 'Target')

In [ ]:
plot_grouped_bar_chart(df, 'eviv3', 'Target')

In [ ]:
plot_grouped_bar_chart(df,'epared2','Target')

In [ ]:
plot_grouped_bar_chart(df,'pisocemento','Target')

Generally it seems easy to classify the relatively wealthy, and hard to classify the relatively poor, even with the highest correlation variables